In [1]:
import pandas as pd
import re
import os

from bs4 import BeautifulSoup

In [2]:
def is_page_empty(bs) -> bool:
    empty_condition = bs.find('h2', class_='text-white font-weight-bold')

    if empty_condition is None:
        return False
    else:
        return True

In [3]:
def parse_salary(salary) -> dict:
    # pomijamy dziwne ogłoszenie firmy Alten
    if salary != '$':
        bounds = re.findall('[0-9]+', salary.replace(' ', ''))

        low_bound = bounds[0]+bounds[1]
        # paskudny hack
        high_bound = bounds[2]+bounds[3] if len(bounds) > 2 else bounds[0]+bounds[1]
        currency = salary.split()[-1]

        return {'low': low_bound,
                'high': high_bound,
                'currency': currency}
    

In [4]:
def parse_city(city) -> dict:
    # pomijamy navalgo z dolarem
    if re.search('(Zdalna)', city) or re.search('\d{2}', city):
         _city = "Zdalna"
         country = 'N/A'
    else:       
        (_city, country) = city.split(',')
        country = country.strip()

    return {'city': _city, 'country': country}

In [5]:
def parse_jobs(jobs) -> list:
    results = []
    
    for job in jobs:
        results.append(parse_job(job))

    return results

In [6]:
def parse_job(job) -> dict:
    job_info = job.find('div', class_='posting-info position-relative d-lg-flex flex-grow-1 align-items-center ng-star-inserted').find_all('span')
    
    salary = job_info[0].text.strip()
    salary_data = parse_salary(salary)
    
    location = job.find('nfj-posting-item-city')
    if location is None:
        location = job_info[1]

    location = location.text.strip()
    location = parse_city(location)

    name = job.find('h3', class_='posting-title__position').text.strip()
    
    company = job.find('span', class_='posting-title__company').text.replace('w', '').strip()

    technology = job.find('a', class_='btn btn-outline-secondary btn-sm text-truncate')
    if technology:
        technology = technology.text.strip()
    else:
        technology = 'N/A'

    return {
        'location': location,
        'salary': salary_data,
        'name': name,
        'company': company,
        'technology': technology
    } 

In [7]:
def get_data(page) -> list:
    results = []

    jobs = [x.parent for x in page.find_all('div', class_='posting-image')]
    page_data = parse_jobs(jobs)
    results += page_data

    return results

In [8]:
data_dir = '../data/raw'
results = []

for entry in os.scandir(data_dir):
    with open(
        os.path.join(data_dir, entry.name),
        encoding='UTF-8') as f:
            html = f.read()

    job = entry.name.split('_')[0]
    bs = BeautifulSoup(html)
    offers = get_data(bs)
    for offer in offers:
        offer['job'] = job
    results += offers

In [9]:
offers

[{'location': {'city': 'Zdalna', 'country': 'N/A'},
  'salary': {'low': '8000', 'high': '15000', 'currency': 'PLN'},
  'name': 'Software Tester',
  'company': 'RST Softare Masters',
  'technology': 'N/A',
  'job': 'test automation'},
 {'location': {'city': 'Gdańsk', 'country': 'POL'},
  'salary': {'low': '8000', 'high': '10000', 'currency': 'PLN'},
  'name': 'Junior Test Automation Engineer',
  'company': 'Plenti',
  'technology': 'N/A',
  'job': 'test automation'},
 {'location': {'city': 'Wrocław', 'country': 'POL\n + 1'},
  'salary': {'low': '13440', 'high': '19320', 'currency': 'PLN'},
  'name': 'Test Automation Engineer (Python)',
  'company': 'Siili',
  'technology': 'N/A',
  'job': 'test automation'},
 {'location': {'city': 'Warsaw', 'country': 'POL'},
  'salary': {'low': '9000', 'high': '12500', 'currency': 'PLN'},
  'name': 'Automation Test Analyst (Java)',
  'company': 'HL Tech',
  'technology': 'N/A',
  'job': 'test automation'},
 {'location': {'city': 'Warsaw', 'country': 'P

In [10]:
df = pd.json_normalize(results)
df.head()

,name,company,technology,job,location.city,location.country,salary.low,salary.high,salary.currency,salary
0,Data Engineer (Microsoft),Onelo S.A.,N/A,data engineer,Zdalna,N/A,13650,17850,PLN,NaN
1,Data Engineer,Phorest,N/A,data engineer,Zdalna,N/A,16773,23962,PLN,NaN
2,Data Engineer,Ringier Axel Springer Tech,N/A,data engineer,Zdalna,N/A,10100,16900,PLN,NaN
3,Scala Big Data Engineer,VirtusLab,N/A,data engineer,Zdalna,N/A,15000,22000,PLN,NaN
4,Software Engineer (Scala) | Big Data,Adform Sp. z o.o.,N/A,data engineer,Zdalna,N/A,18000,21000,PLN,NaN


In [11]:
df.to_csv('../data/interim/job_offers.csv', sep=';', encoding='UTF', index=False)

In [12]:
results

[{'location': {'city': 'Zdalna', 'country': 'N/A'},
  'salary': {'low': '13650', 'high': '17850', 'currency': 'PLN'},
  'name': 'Data Engineer (Microsoft)',
  'company': 'Onelo S.A.',
  'technology': 'N/A',
  'job': 'data engineer'},
 {'location': {'city': 'Zdalna', 'country': 'N/A'},
  'salary': {'low': '16773', 'high': '23962', 'currency': 'PLN'},
  'name': 'Data Engineer',
  'company': 'Phorest',
  'technology': 'N/A',
  'job': 'data engineer'},
 {'location': {'city': 'Zdalna', 'country': 'N/A'},
  'salary': {'low': '10100', 'high': '16900', 'currency': 'PLN'},
  'name': 'Data Engineer',
  'company': 'Ringier Axel Springer Tech',
  'technology': 'N/A',
  'job': 'data engineer'},
 {'location': {'city': 'Zdalna', 'country': 'N/A'},
  'salary': {'low': '15000', 'high': '22000', 'currency': 'PLN'},
  'name': 'Scala Big Data Engineer',
  'company': 'VirtusLab',
  'technology': 'N/A',
  'job': 'data engineer'},
 {'location': {'city': 'Zdalna', 'country': 'N/A'},
  'salary': {'low': '18000